In [1]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

def extract_sift_des(image):
    sift = cv2.ORB_create()
    kp, des = sift.detectAndCompute(image, None)
    return des

def read_bounding_box(path):
    boundary_file = open(path, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    return boundary

transform = transforms.Compose([
    transforms.Resize(260),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [2]:
download_path='C:/Users/IV/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

model = models.efficientnet_b3(pretrained=True)
feat_extractor = model.features #define the feature extractor
layer1 = model.features[:-1]
feat_extractor.eval()  #set the mode as evaluation

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

gallery_features = []
gallery_des = []

for i, gallery_img_no in tqdm(enumerate(gallery_imgs_no)):
    per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
    per_gallery=cv2.imread(per_gallery_name)
    
    # Image pre-processing
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_BGR2YUV)
    per_gallery[:,:,0] = cv2.equalizeHist(per_gallery[:,:,0])
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_YUV2BGR)
    per_gallery = cv2.GaussianBlur(per_gallery,(3,3),0)
    gallery_sift = extract_sift_des(per_gallery)
    per_gallery = Image.fromarray(per_gallery)

    # preprocess the input image
    img_transform = transform(per_gallery) #normalize the input image and transform it to tensor.
    img_transform = torch.unsqueeze(img_transform, 0)
    
    # feature extraction for per gallery
    with torch.no_grad():
        per_gallery_features = model(img_transform)

    gallery_features.append(per_gallery_features)
    gallery_des.append(gallery_sift)



C:\Users\IV\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\IV\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
5000it [33:55,  2.46it/s]


In [3]:
query_features = []
query_des = []
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    
    # crop the image
    boundary = read_bounding_box(queryfilename)
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]

    # Image pre-processing
    query_boundary=cv2.cvtColor(query_boundary,cv2.COLOR_BGR2YUV)
    query_boundary[:,:,0] = cv2.equalizeHist(query_boundary[:,:,0])
    query_boundary = cv2.cvtColor(query_boundary,cv2.COLOR_YUV2BGR)
    query_boundary = cv2.GaussianBlur(query_boundary,(3,3),0)

    query_des.append(extract_sift_des(query_boundary))
    query_boundary = Image.fromarray(query_boundary)
    query_transformed = transform(query_boundary) #normalize the input image and transform it to tensor.
    query_transformed = torch.unsqueeze(query_transformed, 0) #Set batchsize as 1. You can enlarge the batchsize to accelerate.

    with torch.no_grad():
        query_features.append(model(query_transformed))

20it [00:04,  4.13it/s]


In [4]:
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    time_s = time.time()
    dist_record=[]
    gallery_imgs_no_desc=[]
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        sim_score1 = cosine_similarity(query_features[i], gallery_features[j]) 
        sim_score2 = cosine_similarity(query_des[i], gallery_des[j])
        sim_score = sim_score1*0.7 + sim_score2*0.3
        dist_record.append(sim_score)
        # print(sim_score)
        # find the indexes with descending similarity order
        
    descend_index=sorted(range(len(dist_record)), key=lambda k: np.max(dist_record[k]),reverse=True)
    # update the results for one query
    for k in range(len(descend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[descend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top 10 ranked similar image No. are {gallery_imgs_no_desc[0]} {gallery_imgs_no_desc[1]} {gallery_imgs_no_desc[2]} {gallery_imgs_no_desc[3]} {gallery_imgs_no_desc[4]} {gallery_imgs_no_desc[5]} {gallery_imgs_no_desc[6]} {gallery_imgs_no_desc[7]} {gallery_imgs_no_desc[8]} {gallery_imgs_no_desc[9] }')

Processing query part 0: 5000it [00:16, 310.77it/s]
1it [00:24, 24.16s/it]

retrieval time for query 1258 is 24.155112504959106s
For query image No. 1258, the top 10 ranked similar image No. are 2403 4937 3886 1025 3689 230 4383 2941 1835 4696


Processing query part 1: 5000it [00:15, 325.12it/s]
2it [00:47, 23.60s/it]

retrieval time for query 1656 is 23.20803213119507s
For query image No. 1656, the top 10 ranked similar image No. are 2003 839 1227 934 3875 2490 1243 991 2602 4498


Processing query part 2: 5000it [00:15, 313.92it/s]
3it [01:11, 23.65s/it]

...

retrieval time for query 4716 is 25.791189193725586s
For query image No. 4716, the top 10 ranked similar image No. are 1215 2088 1137 3084 306 1815 518 367 4978 3581


Processing query part 18: 5000it [00:17, 279.38it/s]
19it [07:30, 25.30s/it]

retrieval time for query 4929 is 26.23075008392334s
For query image No. 4929, the top 10 ranked similar image No. are 672 1025 3350 931 4072 2782 2704 110 4948 1273


Processing query part 19: 5000it [00:10, 475.81it/s]
20it [07:48, 23.44s/it]

retrieval time for query 776 is 18.500486850738525s
For query image No. 776, the top 10 ranked similar image No. are 204 694 4918 1612 3846 3850 4431 1958 2936 2778


In [5]:
# write the output file following the example
f=open(r'./rank_list_CNN&SIFT.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()